---
# Objectives

1. Convert newick tree to PhyloXML format.
1. Fixes internal node names from an IQTREE tree. 
   - internal nodes by default get named the confidence (ex. "99/100")
   - this notebook instead gives them the name NODEi where i is an integer starting from 0.
   - NODE0 therefore becomes the root.
1. Convert tree to data frame.
1. Combine metadata with tree data frame.
1. Export:
   - Tree (newick, nexus, phyloxml)
   - Tree Dataframe (tsv)
   - Color mapping (tsv)
   - Lat Lon Mapping for Auspice (tsv)
   - Lat Lon Mapping for BEAST (tsv)   
   - Augur JSON
   - Auspice JSON

---
# Setup

## Module Imports

In [1]:
# stats and plotting
import pandas as pd
from matplotlib import colors, gridspec
import math

# Phylogenetics
from Bio import Phylo
import treetime # Clock filter tips

# System misc

import time # Augur export
import os
import copy
from augur import utils, export_v2
import json

# Logging to file
import sys
import io

# Writing python objects to file
import dill

## Variables

In [2]:
from config import *

# Custom script variables
SCRIPT_NAME = "parse_tree"

try:
    WILDCARDS = snakemake.wildcards
    project_dir = os.getcwd()
except NameError:
    WILDCARDS = ["all", "chromosome", "5", "prune"]
    project_dir = os.path.dirname(os.path.dirname(os.getcwd()))
    
READS_ORIGIN = WILDCARDS[0]
LOCUS_NAME = WILDCARDS[1]
MISSING_DATA = WILDCARDS[2]
PRUNE = WILDCARDS[3]


NAME_COL = "sample"

## Input File Paths

In [3]:
config_dir = os.path.join(project_dir, "results", "config")
results_dir = os.path.join(project_dir, "results")
outdir = os.path.join(results_dir, 
                      SCRIPT_NAME,
                      READS_ORIGIN,
                      LOCUS_NAME,
                      "filter{}".format(MISSING_DATA),
                     )

tree_path = os.path.join(
    results_dir, 
    "iqtree", 
    READS_ORIGIN, 
    LOCUS_NAME,
    PRUNE,
    "filter{}".format(MISSING_DATA),    
    "iqtree.treefile",
)

metadata_path = os.path.join(
    results_dir,
    "metadata",
    READS_ORIGIN,  
    "metadata.tsv"
)

aln_path = os.path.join(
    results_dir,
    "snippy_multi",
    READS_ORIGIN,
    LOCUS_NAME,
    PRUNE,
    "filter{}".format(MISSING_DATA),       
    "snippy-multi.snps.aln",
)

auspice_config_path = os.path.join(config_dir, "auspice_config.json")
auspice_remote_dir_path = os.path.join(project_dir, "auspice/")

print("tree_path:", tree_path)
print("metadata_path:", metadata_path)
print("aln path:", aln_path)
print("auspice_config_path:", auspice_config_path)
print("auspice_remote_dir_path:", auspice_remote_dir_path)
print("outdir:", outdir)
# Create output directory if it doesn't exist
if not os.path.exists(outdir):
    os.makedirs(outdir)

tree_path: /mnt/c/Users/ktmea/Projects/plague-phylogeography/results/iqtree/all/chromosome/filter5/iqtree_post.cf.reroot.nwk
metadata_path: /mnt/c/Users/ktmea/Projects/plague-phylogeography/results/metadata/all/metadata.tsv
aln path: /mnt/c/Users/ktmea/Projects/plague-phylogeography/results/snippy_multi/all/chromosome/filter5/snippy-multi.snps.aln
auspice_config_path: /mnt/c/Users/ktmea/Projects/plague-phylogeography/results/config/auspice_config.json
auspice_remote_dir_path: /mnt/c/Users/ktmea/Projects/plague-phylogeography/auspice/
outdir: /mnt/c/Users/ktmea/Projects/plague-phylogeography/results/parse_tree/all/chromosome/filter5


## Import Metadata Dataframe

In [ ]:
# Parse the metadata
metadata_df = pd.read_csv(metadata_path, sep='\t')

# Fix the problem with multiple forms of NA in the table
# Consolidate missing data to the NO_DATA_CHAR
metadata_df.fillna(NO_DATA_CHAR, inplace=True)

# set the metadata index
metadata_df.set_index(NAME_COL, inplace=True)

metadata_df

---
# 1. Convert tree to PhyloXML format

In [ ]:
tmp_xml = os.path.join(outdir, "temp.xml")
try:
    Phylo.convert(tree_path, "newick", tmp_xml, "phyloxml")
except FileNotFoundError:
    Phylo.convert(tree_alt_path, "newick", tmp_xml, "phyloxml")

---
# 2. Fix internal node names

## Read in XML tree

In [ ]:
# Read in the new tree
tree = Phylo.read(tmp_xml, "phyloxml")

# Deepest nodes first
tree.ladderize(reverse=False)

## Fix node names and confidences

In [ ]:
# Counter for integer node names
node_i = 0

# Iterate through the nodes in the tree
for c in tree.find_clades():
    # Check if the name has 'accidentally' become the confidence vals
    ufboot_val = NO_DATA_CHAR    
    scf_val = NO_DATA_CHAR
    # If it's a named node, parse it
    if c.name:
        name_split = c.name.split("/")
        
        # Check if the name has 'accidentally' become the confidence vals
        if len(name_split) > 1:
            # Name the internal node
            c.name = "NODE" + str(node_i)    

            # Assign confidence values     
            ufboot_val = float(name_split[0])
            scf_val = float(name_split[1])
    # If it's not a named node, give it a name
    else:
        c.name = "NODE" + str(node_i) 
        # The one and only confidence is "unknown" and if UFboot
        if len(c.confidences) > 0:
            ufboot_val = c.confidences[0].value
        c.confidences = []
        
    
    ufboot_conf = Phylo.PhyloXML.Confidence(ufboot_val, type="ufboot")
    scf_conf = Phylo.PhyloXML.Confidence(scf_val, type="scf")
    
    c.confidences.append(ufboot_conf)
    c.confidences.append(scf_conf)
        
    # Increment node counter if not a terminal
    if not c.is_terminal():
        node_i += 1

In [ ]:
fig, ax1 = plt.subplots(1, dpi=400)

Phylo.draw(tree,
           axes=ax1, 
           show_confidence=False, 
           label_func = lambda x:'', 
           do_show=False,
           )

---
# 3. Convert tree to dataframe

## Initialize the tree dataframe

In [ ]:
# initialize empty dataframe
tree_df = pd.DataFrame(columns = [
    "name",
    "ufboot", 
    "scf", 
    "branch_length", 
    "node_type",
    "coord_x", 
    "coord_y",
]
)

for c in tree.find_clades():
    if not c.branch_length: c.branch_length = 0.0
    
    node_data = {
        "name" : c.name,
        "ufboot" : [conf.value for conf in c.confidences if conf.type=="ufboot"][0],
        "scf" :  [conf.value for conf in c.confidences if conf.type=="scf"][0],
        "node_type" :  NO_DATA_CHAR,
        "branch_length" :  c.branch_length,
        "coord_x" :  NO_DATA_CHAR,
        "coord_y" :  NO_DATA_CHAR,
    }
    
    tree_df = tree_df.append(node_data, ignore_index=True)

# Set the index to the node name
tree_df.set_index("name", inplace=True)

# Visualize data frame
tree_df

## Add plotting x and y coordinates of each node

In [ ]:
x_posns = get_x_positions(tree)
y_posns = get_y_positions(tree)

# Add x and y coordinates as other attributes
for c in tree.find_clades():
    # x coordinates will be of branch length units
    coord_x = [value for key,value in x_posns.items() if key.name == c.name][0]
    # y coordinates will be arbitrary, based on number of tips
    coord_y = [value for key,value in y_posns.items() if key.name == c.name][0]
    # Add data to tree dataframe
    tree_df.at[c.name, 'coord_x'] = coord_x
    tree_df.at[c.name, 'coord_y'] = coord_y    

# Visualize dataframe
tree_df

## Add node type

In [ ]:
for c in tree.find_clades():   
    # Default Color
    node_type = "internal"
    # Terminal branches will be grey
    if c.is_terminal():
        node_type = "terminal"
    tree_df.at[c.name, "node_type"] = node_type          

tree_df

---
# 5. Combine metadata into tree dataframe

## Parse metadata into df

In [ ]:
# Parse the metadata
metadata_df = pd.read_csv(metadata_path, sep='\t')

# Fix the problem with multiple forms of NA in the table
# Consolidate missing data to the NO_DATA_CHAR
metadata_df.fillna(NO_DATA_CHAR, inplace=True)

# set the metadata index
metadata_df.set_index(NAME_COL, inplace=True)

metadata_df

## Add metadata to tree and tree dataframe

In [ ]:
# Iterate through the different metadata attributes
for attr in metadata_df.columns:
    # Initialize an empty column for the attribute
    tree_df[attr.lower()] = [NO_DATA_CHAR for row in range(0,len(tree_df))]
    # Iterate over the nodes in the tree
    for c in tree.find_clades():
        # Initialize to empty
        attr_val = NO_DATA_CHAR
        # If the node shows up in the metadata
        if c.name in metadata_df.index:
            tree_df.at[c.name, attr.lower()] = metadata_df[attr][c.name]
            attr_val = metadata_df[attr][c.name]         

tree_df

## Add Reference Metadata

In [ ]:
for col in tree_df.columns:
    if col in REF_META:
        tree_df.at["Reference", col] = REF_META[col]

## Make NA values consistent

In [ ]:
# Fix the problem with multiple forms of NA in the table
# Consolidate missing data to the NO_DATA_CHAR
tree_df.fillna(NO_DATA_CHAR, inplace=True)

# ?. Add Metadata As Node Comments

In [ ]:
for c in tree.find_clades():
    c.comment = ""
    for col in tree_df.columns:
        col_val = tree_df[col][c.name]   
        # Test for non ascii
        if type(col_val) == str and not col_val.isascii():
            col_val = col_val.encode("unicode_escape")     
        # Add comment
        if not hasattr(c, "comment") or not c.comment:
            c.comment = "&{}={}".format(col, col_val)
        else:
            c.comment += ",{}={}".format(col, col_val)

---
# 6. Export

## Objects

## Save tree and tree dataframe

In [ ]:
# Dataframe
out_path_df = os.path.join(outdir, SCRIPT_NAME + ".tsv" )
out_path_pickle_df = os.path.join(outdir,  SCRIPT_NAME + ".df.obj" )

tree_df.to_csv(out_path_df, sep="\t")
with open(out_path_pickle_df,"wb") as outfile:
    dill.dump(tree_df, outfile)

# Nexus
out_path_nexus = os.path.join(outdir, SCRIPT_NAME + ".nexus" )
Phylo.write(tree, out_path_nexus, 'nexus', format_branch_length='%1.{}f'.format(BRANCH_LEN_SIG_DIG))

# Dill object
out_path_dill_tree = os.path.join(outdir,  SCRIPT_NAME + ".phylo.obj" )
with open(out_path_dill_tree,"wb") as outfile:
    dill.dump(tree, outfile)
    
# Newick (remove comments)
for c in tree.find_clades(): c.comment = None
out_path_nwk = os.path.join(outdir, SCRIPT_NAME + ".nwk" )
Phylo.write(tree, out_path_nwk, 'newick', format_branch_length='%1.{}f'.format(BRANCH_LEN_SIG_DIG))

## Save lat and lon

Write an output tsv for auspice that is:  
resolution   name   lat    lon

Write an output tsv for beast that is: 
sample lat

Write an output tsv for beast that is: 
sample lon

In [ ]:
latlon_dict_auspice = {"country": {}, "province": {}}
latlon_dict_beast = {}

out_path_lat_lon_auspice = os.path.join(outdir, SCRIPT_NAME + "_lat-lon_auspice.tsv")
out_path_lat_beast = os.path.join(outdir, SCRIPT_NAME + "_lat_beast.tsv")
out_path_lon_beast = os.path.join(outdir, SCRIPT_NAME + "_lon_beast.tsv")

for c in tree.find_clades():
    # For auspice
    # Iterate through each geographic level
    for level in latlon_dict_auspice:
        # Store the location name
        loc = tree_df[level][c.name]
        # If the node has location data and is new
        if loc != NO_DATA_CHAR and loc not in latlon_dict_auspice[level]:
            latlon_dict_auspice[level][loc] = {}
            latlon_dict_auspice[level][loc]["lat"] = tree_df[level + "_lat"][c.name]
            latlon_dict_auspice[level][loc]["lon"] = tree_df[level + "_lon"][c.name]
    
    # For beast
    # skip internal nodes
    if not c.is_terminal(): continue
    # Get latitude and longitude
    lat = tree_df["province_lat"][c.name]
    if lat == NO_DATA_CHAR:
        lat = tree_df["country_lat"][c.name]
    lon = tree_df["province_lon"][c.name]
    if lon == NO_DATA_CHAR:
        lon = tree_df["country_lon"][c.name] 
    
    latlon_dict_beast[c.name] = {"lat": lat, "lon": lon}    

with open(out_path_lat_lon_auspice, "w") as outfile:
    for level in latlon_dict_auspice:
        for loc in latlon_dict_auspice[level]:
            outfile.write(level.lower() + "\t"
                         + loc + "\t"
                         + str(latlon_dict_auspice[level][loc]["lat"]) + "\t"
                         + str(latlon_dict_auspice[level][loc]["lon"]) + "\n"
                 )
            
with open(out_path_lat_beast, "w") as latfile:
    with open(out_path_lon_beast, "w") as lonfile:          
        for sample in latlon_dict_beast:
            latfile.write(sample + "\t" + str(latlon_dict_beast[sample]["lat"]) + "\n")
            lonfile.write(sample + "\t" + str(latlon_dict_beast[sample]["lon"]) + "\n")

# For clarity with auspice        
auspice_latlons_path = out_path_lat_lon_auspice        

## Save Dates

Write an output tsv for beast that is:  
sample    date

In [ ]:
dates_dict_beast = {}
out_path_dates_beast = os.path.join(outdir, SCRIPT_NAME + "_dates_beast.tsv")

for c in tree.find_clades():
    if not c.is_terminal(): continue
    date_bp = str(tree_df["date_bp"][c.name])
    if date_bp == NO_DATA_CHAR: continue
    date_bp = date_bp.lstrip("[").rstrip("]")
    date_bp_split = [float(d) for d in date_bp.split(":")]
    # Calculate the mean
    date_bp_mean = sum(date_bp_split) / len(date_bp_split)
    # Convert to positive
    date_bp_pos = 0 - date_bp_mean
    dates_dict_beast[c.name] = date_bp_pos

with open(out_path_dates_beast, "w") as datefile:    
    for sample in dates_dict_beast:
        datefile.write(sample + "\t" + str(dates_dict_beast[sample]) + "\n")

## Save colors

Write an output tsv that is:
attribute   attribute_val   hex_color

In [ ]:
out_path_colors = os.path.join(outdir, SCRIPT_NAME + "_colors.tsv")
file_colors = open(out_path_colors, "w")

hex_dict = {}

for attr in ATTRIBUTE_LIST:
    # Create the color map
    attr_key = attr.lower()
    hex_dict[attr_key] = {}
    for t in tree.get_terminals():
        attr_val = tree_df[attr][t.name]
        if attr_val not in hex_dict[attr_key] and attr_val != NO_DATA_CHAR:
            hex_dict[attr_key][attr_val] = {}
            
    # Create the custom color map (pyplot)
    cmap = plt.get_cmap(CONT_COLOR_PAL, len(hex_dict[attr_key]))
    # Convert the color map to a list of RGB values
    cmaplist = [cmap(i) for i in range(cmap.N)]
    # Convert RGB values to hex colors
    attr_hex = [colors.to_hex(col) for col in cmaplist]
    
    # Assign colors to value
    for attr_val, attr_col in zip(hex_dict[attr_key], attr_hex):
        hex_dict[attr_key][attr_val] = attr_col   

for attr_key in hex_dict:
    for attr_val in hex_dict[attr_key]:
        file_colors.write(str(attr_key) + "\t" + str(attr_val) + "\t" + str(hex_dict[attr_key][attr_val]) + "\n")

file_colors.close()        

# For clarity with auspice
auspice_colors_path = out_path_colors

## Augur JSON

  - alignment (empty)
  - input_tree (tree_path)
  - nodes (node_dict)


In [ ]:
augur_dict = augur_export(
    tree_path=out_path_nwk, 
    aln_path=aln_path, 
    tree=tree, 
    tree_df=tree_df, 
    color_keyword_exclude=["color", "coord", "lat", "lon"],
    type_convert = {
        "branch_number" : (lambda x : str(x))
    },
)

out_path_augur_json = os.path.join(outdir, SCRIPT_NAME + "_augur.json" )
utils.write_json(data=augur_dict, file_name=out_path_augur_json, indent=JSON_INDENT)

print(augur_dict["nodes"]["Reference"])

## Auspice JSON

Manual edits of https://github.com/nextstrain/augur/blob/master/augur/export_v2.py

This can then be used for auspice via:

```
HOST="localhost" auspice view --datasetDir .
```

In [ ]:
auspice_dict = auspice_export(
    tree=tree, 
    augur_json_paths=[out_path_augur_json], 
    auspice_config_path=auspice_config_path, 
    auspice_colors_path=auspice_colors_path,
    auspice_latlons_path=auspice_latlons_path, 
    )

# Write outputs - For Local Rendering
out_path_auspice_local_json = os.path.join(outdir, SCRIPT_NAME + "_auspice.json" )
utils.write_json(data=auspice_dict, file_name=out_path_auspice_local_json, indent=JSON_INDENT, include_version=False)
export_v2.validate_data_json(out_path_auspice_local_json)
print("Validation successful for local JSON.")

# Write outputs - For Remote Rendering
#out_path_auspice_remote_json = os.path.join(auspice_remote_dir_path, AUSPICE_PREFIX + SCRIPT_NAME.replace("_","-") + ".json" )
#utils.write_json(data=auspice_dict, file_name=out_path_auspice_remote_json, indent=JSON_INDENT, include_version=False)
#export_v2.validate_data_json(out_path_auspice_remote_json)
print("Validation successful for remote JSON.")

---
## Cleanup

In [ ]:
os.remove(tmp_xml)